In [ ]:
import pandas as pd
import logging

# Log fayl manzili
log_path = r"C:\Users\Rasulbek907\Desktop\Project_8\Log\error_analysis.log"

# Log sozlamalari
logging.basicConfig(
    filename=log_path,
    filemode='a',  # Append mode
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

try:
    logging.info("CSV fayl o'qilmoqda:...")
    df = pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Project_8\Data\Feature_Selection\Filtered_Features.csv")
    logging.info(f"Fayl muvaffaqiyatli o'qildi. Satırlar soni: {len(df)} ustunlar soni: {len(df.columns)}")
except Exception as e:
    logging.error(f"CSV faylni o'qishda xatolik: {e}")

In [ ]:
df.info()

In [ ]:
import shap
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# --- Ma'lumotni tayyorlash ---
X = df.drop(columns=['Unnamed: 0', 'cluster'])  # features
y = df['cluster']  # target

# --- Train/Test split (agar kerak bo'lsa) ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Model yaratish va fit qilish ---
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# --- SHAP explainer yaratish ---
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

# --- Summary plot ---
plt.title("SHAP Summary Plot")
shap.summary_plot(shap_values, X_test, plot_type="bar")

# --- Waterfall plot uchun bitta misol (testdagi birinchi sample) ---
# RandomForestClassifier ko'pklass bo'lgani uchun shap_values har bir class uchun alohida bo'ladi
# Faraz qilamiz class 0 uchun
class_index = 0  
shap.plots._waterfall.waterfall_legacy(explainer.expected_value[class_index], shap_values[class_index][0], X_test.iloc[0])
plt.show()